In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import time

from poliastro.twobody.events import LithobrakeEvent
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.util import norm
from poliastro.constants import R_earth, R_mean_earth, GM_earth
from poliastro.core.elements import coe2rv
from poliastro.twobody.propagation import CowellPropagator

from astropy import units as u
from astropy.time import Time

from datetime import datetime, timedelta
from sso_inc import inc_from_alt, raan_from_ltan,angle_between
from perturbations import perturbations_atm_J2, pertubations_coesa_high, pertubations_coesa_low, acc_max_vs_min 


from scipy.optimize import line_search

toc = time.time()
## Orbit
h = 300
start_date = datetime(2024,1,1,9,0,0)
ltan = 22.5

a = (R_earth.value/1000 + h) << u.km
ecc = 1e-6 << u.one
inc = inc_from_alt(h,ecc)[0] << u.deg   
raan = raan_from_ltan(Time(val=datetime.timestamp(start_date), format='unix'),ltan) << u.deg
argp = 1e-6 << u.deg
nu = 1e-6 << u.deg

epoch = Time(val=start_date.isoformat(), format='isot')

reference_orbit = Orbit.from_classical(
    Earth,
    a,
    ecc,
    inc,
    raan,
    nu,
    argp,
    epoch
    )
trailing_orbit = Orbit.from_classical(
    Earth,
    a+(5<<u.km),
    ecc,
    inc,
    raan,
    nu+(0<<u.deg),
    argp,
    epoch
    )

# Prop
start_date_ts = datetime.timestamp(start_date)
stop_date_ts = datetime.timestamp(start_date + timedelta(hours = 24*7*5))
sample_num = 9*1*16*7*2
timestamps = np.linspace(start_date_ts, stop_date_ts, sample_num)
time_step = (timestamps[1]-timestamps[0]) << u.s
refsmalist = []
trailsmalist = []

assignment = 10

ref_vel = []
trail_vel = []

elapsedsecs = []
secs = 0

rmag_ref = []
rmag_trail = []

vmag_ref = []
vmag_trail = []

angle_list = []

# ang_vel_ref_list = []
# ang_vel_trail_list = []
ang_vel_list = []

fig, ax = plt.subplots(2, 3, figsize=(22,9), squeeze=False) 

phi_dot_ref = reference_orbit.n.to(u.deg/u.s)
phi_dot_tra = trailing_orbit.n.to(u.deg/u.s)
phi_dot_dot = (acc_max_vs_min(trailing_orbit.r.value, trailing_orbit.v.value)) << (u.deg/(u.s**2))
delta_phi_dot = phi_dot_ref - phi_dot_tra

delta_theta = 10 - angle_between(trailing_orbit.r.value,reference_orbit.r.value)
delta_theta_dot = 0 - delta_phi_dot

def obj_func(t_a):
    return (phi_dot_dot)*(t_a**2) + (-delta_phi_dot*2)*(t_a) + (delta_theta - delta_theta_dot*(delta_phi_dot+0.5*delta_theta_dot)/)
